# Reinforcement Learning - Cheatsheet

In [ ]:
# !pip install stable-baselines3[extra]

In [1]:
import os
import gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.evaluation import evaluate_policy

## Cartpole

### Env

In [2]:
ENV_NAME = 'CartPole-v0'

#### Testing

In [3]:
env = gym.make(ENV_NAME)

episodes = 5
for ep in range(1, episodes+1):
    state = env.reset() # initial observations for the env
    done = False
    score = 0
    while not done:
        env.render()
        action = env.action_space.sample()
        # there are two spaces - action, observation
        # here, observation_space is [cart_pos, cart_vel, pole_angle, pole_angvel]
        # action space: 0 - left, 1 - right
        n_state, reward, done, info = env.step(action)
        # n_state is observation for next state
        score += reward
    
    print(f"Ep #{ep} - Score: {score}")
env.close()

Ep #1 - Score: 17.0
Ep #2 - Score: 63.0
Ep #3 - Score: 13.0
Ep #4 - Score: 32.0
Ep #5 - Score: 18.0


In [3]:
env = gym.make(ENV_NAME)
env = DummyVecEnv([lambda: env])

### Model

There are two types of RL learning algorithms
1. Model Free
    - Uses only the current state
2. Model Bases
    - Tries to predict and use future states

#### Init

In [4]:
LOG_PATH = os.path.join('training', 'logs', ENV_NAME)

In [5]:
# new
# MlpPolicy - Multilayer Perceptron
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH) # creating a new model

Using cpu device


In [ ]:
# load
MODEL_PATH = os.path.join('training', 'saved_models', ENV_NAME, 'v1')
model = PPO.load(MODEL_PATH, env=env) # for loading saved model

In [ ]:
PPO??

#### Learning

In [ ]:
model.learn(total_timesteps=20000)

#### Saving

In [ ]:
model.save(MODEL_PATH)
del model

#### Eval

In [8]:
evaluate_policy(model, env, n_eval_episodes=10, render=True)

(200.0, 0.0)

In [9]:
env.close()

In [10]:
episodes = 5
for ep in range(1, episodes+1):
    obs = env.reset()
    done = False
    score = 0
    while not done:
        env.render()
        action, _ = model.predict(obs)
        # _ holds next_state which is None here
        # used in recurrent policies?
        obs, reward, done, info = env.step(action)
        score += reward
    print(f"Ep #{ep} - Score: {score}")
env.close()

Ep #1 - Score: [37.]
Ep #2 - Score: [144.]
Ep #3 - Score: [109.]
Ep #4 - Score: [192.]
Ep #5 - Score: [200.]


##### Tensorboard

In [ ]:
# !tensorboard --logdir={LOG_PATH}

##### Callbacks

In [6]:
from stable_baselines3.common.callbacks import EvalCallback, StopTrainingOnRewardThreshold

In [7]:
stop_callback = StopTrainingOnRewardThreshold(reward_threshold=200, verbose=1)

eval_callback = EvalCallback(
    env,
    callback_on_new_best=stop_callback,
    eval_freq=10000,
    best_model_save_path=os.path.join('training', 'saved_models', ENV_NAME, input()),
    verbose=1
)

model.learn(total_timesteps=20000, callback=eval_callback)

Logging to training/logs/CartPole-v0/PPO_1
-----------------------------
| time/              |      |
|    fps             | 1222 |
|    iterations      | 1    |
|    time_elapsed    | 1    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 1270        |
|    iterations           | 2           |
|    time_elapsed         | 3           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009395549 |
|    clip_fraction        | 0.113       |
|    clip_range           | 0.2         |
|    entropy_loss         | -0.685      |
|    explained_variance   | 0.00607     |
|    learning_rate        | 0.0003      |
|    loss                 | 8.08        |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0169     |
|    value_loss           | 53.9        |
---------------------------------

/home/kid-116/.local/lib/python3.8/site-packages/stable_baselines3/common/evaluation.py:65: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=200.00 +/- 0.00
Episode length: 200.00 +/- 0.00
------------------------------------------
| eval/                   |              |
|    mean_ep_length       | 200          |
|    mean_reward          | 200          |
| time/                   |              |
|    total_timesteps      | 10000        |
| train/                  |              |
|    approx_kl            | 0.0073796962 |
|    clip_fraction        | 0.063        |
|    clip_range           | 0.2          |
|    entropy_loss         | -0.601       |
|    explained_variance   | 0.319        |
|    learning_rate        | 0.0003       |
|    loss                 | 34.6         |
|    n_updates            | 40           |
|    policy_gradient_loss | -0.0155      |
|    value_loss           | 71.1         |
------------------------------------------
New best mean reward!
Stopping training because the mean reward 200.00  is above the threshold 200


##### Custom Policy

In [ ]:
net_arch = [dict(pi=[128, 128, 128, 128], vf=[128, 128, 128, 128])]
model = PPO('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH, policy_kwargs={'net_arch': net_arch})

model.learn(total_timesteps=20000, callback=eval_callback)

##### Another Algo

In [ ]:
from stable_baselines3 import DQN

model = DQN('MlpPolicy', env, verbose=1, tensorboard_log=LOG_PATH)

model.learn(total_timesteps=20000)